In [26]:
from utils.loading import load_model

In [2]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)


2.5.0+cu124
0.20.0+cu124


In [3]:
import torch
model = torch.hub.load('pytorch/vision', 'resnet50', pretrained=True)

Using cache found in C:\Users\Nech/.cache\torch\hub\pytorch_vision_main
c:\Users\Nech\anaconda3\envs\mineralnet\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Nech\anaconda3\envs\mineralnet\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [60]:
import toml
model_config_path = './configs/models/model_template.toml'
model_config = toml.load(model_config_path)
model_config

{'from': 'pytorch',
 'backbone': {'repo_or_dir': 'pytorch/vision',
  'model': 'resnet18',
  'pretrained': True},
 'module_swap': [{'target': 'fc',
   'new_module': [{'torch_module': 'Linear',
     'args': {'in_features': 2, 'out_features': 10}},
    {'torch_module': 'Linear',
     'args': {'in_features': 2, 'out_features': 10}}]}]}

In [62]:
from typing import Union, List, Dict

import torch.nn as nn
import toml


def load_model(toml_path):
    """load pretrained model from toml config with possible modification"""
    config = toml.load(toml_path)
    model = None

    # load model from model repo
    if 'torch' in config['from']:
        import torch
        model = torch.hub.load(
            **config['backbone'])
    else:
        print('Not implemented yet')

    # modify model
    if 'module_swap' in config:
        for swap in config['module_swap']:
            new_module = load_module(swap['new_module'])
            setattr(model, swap['target'], new_module)
    return model

def load_module(
    module_config: Union[Dict, List[Dict]]
):
    """load module defined in toml config"""
    if isinstance(module_config, dict):
        module_class = getattr(nn, module_config['torch_module'])
        module = module_class(**module_config['args'])
        return module

    return nn.ModuleList([load_module(module) for module in module_config])

In [63]:
model = load_model(model_config_path)


Using cache found in C:\Users\Nech/.cache\torch\hub\pytorch_vision_main


In [25]:
module